In [28]:
import numpy as np
import sys
sys.path.insert(0, '/home/ccj/GCNet/src')
sys.path.insert(0, '/home/ccj/GCNet/src/cpp/featextract/Debug/')
sys.path.insert(0, '/home/ccj/GCNet/src/cpp/matchers/Debug/')

#from skimage.transform import rescale
import pfmutil as pfm
from generator import *

def get_residual(isRescale = True, anti_aliasing = True, s_idx = 0):
    imgs = ['Adirondack', 'ArtL','Jadeplant','Motorcycle','MotorcycleE',
            'Piano','PianoL','Pipes','Playroom','Playtable',
            'PlaytableP','Recycle','Shelves','Teddy','Vintage',]

    root = "/media/ccjData2/"
    dataH_dir = root + "datasets/MiddleBury/MiddEval3/trainingH/All/"
    dataQ_dir = root + "datasets/MiddleBury/MiddEval3/trainingQ/All/"
    result_dir = "/home/ccj/GCNet/results/tmp/"
    #for i in range(0, len(imgs)):
    for i in range(0, 1):
        """H size """ 
        ldata = dataH_dir + "image_0/" + imgs[i] + ".png"
        #print "ldata = {}".format(ldata)
        rdata = dataH_dir + "image_1/" + imgs[i] + ".png"
        ddata = dataH_dir + "dispGT0/" + imgs[i] + ".pfm"
        imgl  = cv2.imread(ldata, 0).astype(np.uint8)
        imgr  = cv2.imread(rdata, 0).astype(np.uint8)
        
        displ = pfm.load(ddata)
        displ[displ == np.inf] = 0.
        ndisp = np.amax(displ)
        
        ds_scale = .5
        
        if isRescale:
            imgl, imgr = down_sampling_input(ds_scale, imgl, imgr, anti_aliasing)
            displ = skimage.transform.rescale(image = displ, scale = ds_scale, multichannel = False, anti_aliasing= anti_aliasing, mode = 'constant', preserve_range = True).astype(np.float32)
            result_dir = os.path.join(result_dir, 'rescale') 
            print "Calling rescale() func ..."
        else:
            imgl = imgl[s_idx::2,s_idx::2]
            imgr = imgr[s_idx::2,s_idx::2]
            displ = displ[s_idx::2,s_idx::2]
            result_dir = os.path.join(result_dir, 'step-idx-{}'.format(s_idx))
            print "Calling [{}::2] indices access ...".format(s_idx)    
        
        """Q size """ 
        ldata = dataQ_dir + "image_0/" + imgs[i] + ".png"
        rdata = dataQ_dir + "image_1/" + imgs[i] + ".png"
        ddata = dataQ_dir + "dispGT0/" + imgs[i] + ".pfm"
        
        imgQl = cv2.imread(ldata, 0).astype(np.uint8)
        imgQr = cv2.imread(rdata, 0).astype(np.uint8)
        dispQl= pfm.load(ddata)
        dispQl[dispQl == np.inf] = 0.

        #resi = abs(imgQl - imgl)
        N = float(imgQl.shape[0]*imgQl.shape[1])
        rl = np.sum(abs(imgl - imgQl)) / N
        rr = np.sum(abs(imgr - imgQr)) / N
        rd = np.sum(abs(displ - dispQl)) / N
        
        N_valid_l  = np.sum(np.where(abs(imgQl - imgl) > 0 , 1, 0))
        N_valid_r  = np.sum(np.where(abs(imgQr - imgr) > 0 , 1, 0))
        N_valid_d  = np.sum(np.where(abs(dispQl- displ)> 0 , 1, 0))

        rl_valid = np.sum(abs(imgl - imgQl)) / N_valid_l
        rr_valid = np.sum(abs(imgr - imgQr)) / N_valid_r
        rd_valid = np.sum(abs(displ - dispQl)) / N_valid_d

        print "Image {}, ndisp = {}, anti_aliasing = {}".format(imgs[i], ndisp, "True" if anti_aliasing else "False")
        print "L1 residual avg err:        rl = {}, rr = {}, rd = {}".format(rl, rr, rd)
        print "L1 residual avg valid err : rl_valid = {}, rr_valid = {}, rd_valid = {}".format(rl_valid, rr_valid, rd_valid)
        
        if not os.path.exists(result_dir):
            os.makedirs(result_dir)
            print 'makedirs {}'.format(result_dir)
            
        cv2.imwrite(os.path.join(result_dir, 'imglH_0.5.png'), imgl)
        cv2.imwrite(os.path.join(result_dir, 'imglQ.png'), imgQl)
        cv2.imwrite(os.path.join(result_dir, 'resi-imgl.png'), abs(imgQl - imgl))
        
        cv2.imwrite(os.path.join(result_dir, 'imgrH_0.5.png'), imgr)
        cv2.imwrite(os.path.join(result_dir, 'imgrQ.png'), imgQr)
        cv2.imwrite(os.path.join(result_dir, 'resi-imgr.png'), abs(imgQr - imgr))
        
        #cv2.imwrite(result_dir + 'resi-err125-imgl.png', np.where(resi < 125, 0, resi))
        pfm.save(os.path.join(result_dir, 'displH_0.5.pfm'), displ)
        pfm.save(os.path.join(result_dir, 'displQ.pfm'), dispQl)
        pfm.save(os.path.join(result_dir,'resi-displ.pfm'), abs(displ - dispQl))
        #pfm.show(abs(imgQl - imgl ).astype(np.float32))
        #cv2.imwrite(result_dir + 'resi-imgl.png', abs(imgQl - imgl ).astype(np.uint8))
        #pfm.save(result_dir + 'resi-imgl.pfm', abs(imgQl - imgl ).astype(np.float32))
        

if __name__ == '__main__':
    
    get_residual(isRescale = True, anti_aliasing = True)
    print "\n"
    get_residual(isRescale = True, anti_aliasing = False)
    print "\n"
    get_residual(isRescale = False, s_idx = 0)
    print "\n"
    get_residual(isRescale = False, s_idx = 1)
    

Calling rescale() func ...
Image Adirondack, ndisp = 126.092956543, anti_aliasing = True
L1 residual avg err:        rl = 129.080897879, rr = 129.432760131, rd = 28.442354996
L1 residual avg valid err : rl_valid = 215.993844766, rr_valid = 217.471609808, rd_valid = 28.4504237038


Calling rescale() func ...
Image Adirondack, ndisp = 126.092956543, anti_aliasing = False
L1 residual avg err:        rl = 106.249494564, rr = 106.356329747, rd = 28.4600873843
L1 residual avg valid err : rl_valid = 206.408645087, rr_valid = 207.387738452, rd_valid = 28.5145337715


Calling [0::2] indices access ...
Image Adirondack, ndisp = 126.092956543, anti_aliasing = True
L1 residual avg err:        rl = 84.6949748405, rr = 84.1742379145, rd = 28.5153091023
L1 residual avg valid err : rl_valid = 126.496166815, rr_valid = 126.307463806, rd_valid = 28.597376558
makedirs /home/ccj/GCNet/results/tmp/step-idx-0


Calling [1::2] indices access ...
Image Adirondack, ndisp = 126.092956543, anti_aliasing = True
L

---

The residual error $e= I_{l1} - I_{l2}$ is shown as below <br> <img style="float: right;" src="files/resi-imgl.png">

---